# Keras for Large Datasets 
---
- generator
- if we have a huge dataset to use lstm we can split them into smaller chunks

In [1]:
import numpy as np


def data_gen(df, batch_size):
    while True:
        x_batch = np.zeros((batch_size, 3, 224, 224))
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(df['url']/batch_size)):
            b = 0
            for m, k in zip(df['url'].values[j*batch_size:(j+1)*batch_size], 
                            df['class'].values[j*batch_size:(j+1)*batch_size]):
                x_batch[b] = m
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)


model.fit_generator(generator=data_gen(df_train, batch_size=batch_size), 
                    steps_per_epoch=len(df_train) // batch_size, epochs=epochs)

NameError: name 'model' is not defined